### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation, meaning the application needs some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

In this guide we focus on adding logic for incorporating historical messages. Further details on chat history management is covered in the previous videos.

We will cover two approaches:

- Chains, in which we always execute a retrieval step;
- Agents, in which we give an LLM discretion over whether and how to execute a retrieval step (or multiple steps).


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")
llm=ChatGroq(model="Llama3-8b-8192")
llm

In [ ]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

create stuff document chain is used for text summarization where it comibnes all the documents first and send to prompt where it get forward to llm

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
import bs4
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content","post-title","post-header") )),)
docs=loader.load()
docs

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
db=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=db.as_retriever()
retriever

In [ ]:
#prompt template
system_prompt=(
    """

    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer
    the question. If you don't know the answer, say that you
    don't know. Use three sentences maximum and keep the
    answer concise."
    \n\n
    {context}
    """
)
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}")
    ]
)
prompt

now upto this

-->we have embedded our website content into vector store and also created a retriever

-->created a prompt through which a system will tell how the model(or llm) should work
   and also provided functionality for input by the user

now time to create the chain through which

-->input will be taken
-->collection of the document chunks reqd and both will be sent to prompt then to llm


In [ ]:
ques_ans_chain=create_stuff_documents_chain(llm,prompt) #prompt->llm just sign of function
ragchain=create_retrieval_chain(retriever,ques_ans_chain)
ragchain

In [ ]:
response=ragchain.invoke({"input":"what is Self-Refletion"})
response

In [ ]:
response['answer']

###now our llm is working fine but all this is going on without any chat history
so again designing a llm with a chat history

ADDING CHAT HISTORY

now for ex

Human who is elon musk

Ai    a buisness man and owner of spacex,starlink....

Human What is his brother name?

now at this point for AI without chat history it's confusing about his brother name

so when i say what his name then two things is going on

--> firstly chat history is getting save

--> using a session id chatbot find the chat then the incoming query from user is internally   re-written for the chatbot w.r.t. history such that chatbot is able to recall everything   and user is just getting a response....
    for them this backend is unvisible
    now for ai the question after reformulating is "what is Elonk musk brother's name?

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
contextualize_qa_system_prompt=(
    """
    Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is.
    """
)
contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_qa_system_prompt),
        MessagesPlaceholder("chathistory"),
        ("human","{input}")
    ]
)

now in prev code we designed how the working will be done

now the code of triggering the working of prev code

In [ ]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

## contexalize_q_prompt -> retriever ->llm

now chat history along with question reformation is done

so designing how every msg sent to AI will get the answer

In [ ]:
## Prompt Template to start the chat
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chathistory"),
        ("human", "{input}"),
    ]
)

so now finally

--> a question asked by user whether a new chat or previous one asking one more ques

----> firstly activates history_aware_retre where  if history exists

        ques is modified acc to history

        retrive of related documents

        but this time llm does not generates answer instead

        goes for qa_prompt act according to system prompt

        then generates answer accordingly

        and for history

        chat_history.extend([

        HumanMessage(content="Tell me more about it?"),

        AIMessage(content=response["answer"])

])


In [ ]:
ques_ans_chain=create_stuff_documents_chain(llm,qa_prompt)
ragchain=create_retrieval_chain(history_aware_retriever,ques_ans_chain)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
q1="What is Self-Refletion?"
r1=ragchain.invoke({"input":q1,"chathistory":chat_history})
print(r1["answer"])

chat_history.extend(
    [
        HumanMessage(content=q1),
        AIMessage(content=r1['answer'])
    ]
)
q2="what are we even talking about? "
r2=ragchain.invoke({"input":q2,"chathistory":chat_history})
print(r2["answer"])

In [ ]:
chat_history

Now upto this we have built a working chatbot with history and we were doing the saving of chat history mannualy


Now let's design a chatbot history working just by changing the way of saving history only

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#just a function signature it first get history then perform ragchain
conversational_rag_chain = RunnableWithMessageHistory(
    ragchain,                                          ## see working is same
    get_session_history,
    input_messages_key="input",
    history_messages_key="chathistory",
    output_messages_key="answer",
)

In [ ]:
response=conversational_rag_chain.invoke({"input":"what is Task Decomposition?"},config={"configurable":{"session_id":"abc123"}})
response["answer"]

In [ ]:
store

In [ ]:
response=conversational_rag_chain.invoke({"input":"what are common techniques to do it?"},config={"configurable":{"session_id":"abc123"}})
response["answer"]

In [ ]:
store

In [ ]:
for session_id, history in store.items():
    print(f"Session ID: {session_id}")
    for msg in history.messages:
        if isinstance(msg, HumanMessage):
            print("Q:", msg.content)
        elif isinstance(msg, AIMessage):
            print("A:", msg.content)
